In [ ]:
%matplotlib notebook

### Animowane wykresy. 

Pierwszym ćwiczeniem polegać będzie na przygotowaniu animowanego wykresu liniowego, który zobrazuje stopy procentowe i inflację. Animację można znaleźć [tutaj](https://twitter.com/PIE_NET_PL/status/1468602618192801793?s=20). 

Aby ją stworzyć wykorzystamy dane z pliku CSV zapisane na Github:

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/jakubrybacki/Karowa_Python_Introduction/main/Zajecia%2010/chart1.csv")
df.info()

Na początku wykonajmy kilka ćwiczeń związanych z pakietem Pandas 
* Python nie rozpozna z automatu, że tekst w zakładce *data* jest datą - trzeba mu to wskazać
* Nagłówki kolumn są skonmplikowane - przekształcmy je do jednowyrazowych

In [ ]:
# Ustawiam format daty
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=False, format='%d/%m/%Y', errors='ignore')

# Jednowyrazowe nagłówki
df["NBP"] = df["Stopa NBP (%)"]
df["CPI"] = df["CPI (%r/r)"]

# Zachowam oryginalne kolumny w innym DataFrame - wykrozystam je potem do stworzenia ładnej legendy
df2 = df[["Stopa NBP (%)", "CPI (%r/r)"]]

# Usuwam kolumny z pełnymi nazwami
df.drop("Stopa NBP (%)", axis='columns', inplace=True)
df.drop("CPI (%r/r)", axis='columns', inplace=True)

# Wydruk informacji po zmianach kolumn
df.info()

Stworzenie animacji zaczynamy od przygotowania nowego wykresu - aby to zrobić potrzebujemy biblioteki *matplotlib.pyplot*:

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.animation as ani

# Tworzy nowy wykres
fig = plt.figure()

# Kolene komendy wykonują modyfikacje wykresu - nie potrzeba odwoływać się bezpośrednio do zmiennych. 
# W poleceniach na dole ustawiamy tytuł oraz marginesy wokół przestrzeni wykresu: 
plt.title("Stopa procentowa NBP i inflacja (%)", loc='center', fontweight="bold",
          fontname="Arial Black")
plt.subplots_adjust(bottom = 0.2, top = 0.9) 

Na wykresie chcemy wyświetlać daty w odpowiednim formacie - najlepiej miesiąc i rok, oba dwucyfrowo.

**Uwaga:** Pełny listing formatów dostepny jest [tutaj](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).

Do zaimplementowania potrzebna będzie kolejna biblioteka: 


In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%m/%y")

Do stworzenia właściwej animacji potzebna jest funkcja, która opowie co rysować w poszczególnych  

In [ ]:
def animeFunction(i = int):
    # Tworzy wykres liniowy gdzie na osi y jest stopa NBP, na osi x data - bierzemy wiersze od pierwszego do i-tego
    # Kolor lini przekazywany jest jako format RGB w kodzie szesnastkowym. 
    plt.plot(df[:i].Data, df[:i].NBP, label = "Stopa NBP (%)", color = "#D22630", 
             scaley=True, scalex=True,)
    plt.plot(df[:i].Data, df[:i].CPI, label = "CPI (%r/r)", color = "#7F7F7F")
    
    # Jednorazowo - na samym początku narysuj legendę:
    if i==0:
        plt.legend(df2.columns, loc = "upper left")
    
    # Ustaw format daty 
    plt.gca().xaxis.set_major_formatter(date_form)


Warto wykonać poszczególną funkcję, aby wiedzieć co generuje:

In [ ]:
animeFunction(14)

Do wygenerowania właściwej animacji służy nam pojedyncza komenda *FuncAnimation*. Tu wykorzystamy jej opisując 3 argumenty:
* fig - wykres na którym chcemy rysować. Ten od którego zaczynaliśmy kody.
* func - funkcja generująca animację, którą przygotowaliśmy w bloku wcześniej
* frames - lista z wartościami dla funkcji animującej, która opisze zachowanie w kolejnych klatkach
* interval - odstęp czasowy między klatkami

In [ ]:
animator = ani.FuncAnimation(fig, func = animeFunction, frames = range(0,len(df)), interval = 1_000)

Aby wyświetlać program w notebooku potrzebny jest kolejny import biblioteki. 

Uwaga: Te komendy będą zupełnie inaczej działać w Jupyter Lab, Colab i Jupyter Notebook

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

animator

Żeby zapisać animację jako gif potrzebna jest instalacja *ImageMagick*. Po instalacji wystarczy wykonać dwie komendy:

In [ ]:
writergif = ani.ImageMagickFileWriter()
animator.save("NBP_CPI.gif", writer = writergif)

Kolejne przykłady pokażę w IDE - Spyder